### 第二部分

训练前的准备

In [1]:
import pandas as pd

# 读取数据
train = pd.read_csv("../dataset/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("../dataset/testData.tsv", header=0, delimiter="\t", quoting=3)
unlabeled_train = pd.read_csv("../dataset/unlabeledTrainData.tsv", header=0, \
                              delimiter="\t", quoting=3)
# 验证数据
print(f"Read {train['review'].size} labeled train reviews, \
      {test['review'].size} labeled test reviews, \
      and {unlabeled_train['review'].size} unlabeled train reviews.")

Read 25000 labeled train reviews,       25000 labeled test reviews,       and 50000 unlabeled train reviews.


In [2]:
# 类似part1, 我们需要清理数据
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re

def review_to_wordlist( review, remove_stopwords=False ):
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    # 2. Remove non-letters&number
    # 教程说最好连数字也不要删除， 只需要稍微修改正则表达式的匹配规则即可
    review_text = re.sub("[^a-zA-Z0-9]", " ", review_text)
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    # 5. Return a list of words
    return words
s = "We are good friend 111 234."
print(review_to_wordlist(s))

d:\ANACONDA\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
d:\ANACONDA\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-

['we', 'are', 'good', 'friend', '111', '234']


Word2Vec的需要的输入格式是以list为单位的句子, 我们采用nltk中的punkt来分割句子.请务必区分list.append()和 += 的区别

In [3]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
# 下载完后注释掉
nltk.download()   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [4]:
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)


Parsing sentences from training set


d:\ANACONDA\lib\site-packages\bs4\__init__.py:272: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
d:\ANACONDA\lib\site-packages\bs4\__init__.py:272: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
d:\ANACONDA\lib\site-packages\bs4\__init__.py:335: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences from unlabeled set


d:\ANACONDA\lib\site-packages\bs4\__init__.py:335: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
d:\ANACONDA\lib\site-packages\bs4\__init__.py:335: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
d:\ANACONDA\lib\site-packages\bs4\__init__.py:272: UserWarning: "b'... ...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
d:\ANACONDA\lib\site-packages\bs4\__init__.py:272: UserW

In [5]:
# 查看一下处理后的结果
print(len(sentences))
print(sentences[0])
print(sentences[1])

795538
['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']
['maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent']


和part1的结果相比, part2的结果中含有stopwords, 并保留有相对顺序说明words2vec在分析时考虑了上下文

确定参数并训练模型

In [6]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            vector_size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2025-04-15 10:23:15,501 : INFO : collecting all words and their counts
2025-04-15 10:23:15,501 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2025-04-15 10:23:15,533 : INFO : PROGRESS: at sentence #10000, processed 227240 words, keeping 18038 word types
2025-04-15 10:23:15,559 : INFO : PROGRESS: at sentence #20000, processed 454577 words, keeping 25324 word types
2025-04-15 10:23:15,593 : INFO : PROGRESS: at sentence #30000, processed 675275 words, keeping 30478 word types
2025-04-15 10:23:15,637 : INFO : PROGRESS: at sentence #40000, processed 903015 words, keeping 34863 word types
2025-04-15 10:23:15,671 : INFO : PROGRESS: at sentence #50000, processed 1123504 words, keeping 38329 word types


Training model...


2025-04-15 10:23:15,708 : INFO : PROGRESS: at sentence #60000, processed 1346265 words, keeping 41338 word types
2025-04-15 10:23:15,748 : INFO : PROGRESS: at sentence #70000, processed 1570739 words, keeping 43986 word types
2025-04-15 10:23:15,783 : INFO : PROGRESS: at sentence #80000, processed 1791249 words, keeping 46400 word types
2025-04-15 10:23:15,807 : INFO : PROGRESS: at sentence #90000, processed 2016723 words, keeping 48869 word types
2025-04-15 10:23:15,856 : INFO : PROGRESS: at sentence #100000, processed 2239896 words, keeping 50980 word types
2025-04-15 10:23:15,892 : INFO : PROGRESS: at sentence #110000, processed 2460901 words, keeping 52890 word types
2025-04-15 10:23:15,933 : INFO : PROGRESS: at sentence #120000, processed 2684304 words, keeping 54967 word types
2025-04-15 10:23:15,973 : INFO : PROGRESS: at sentence #130000, processed 2911246 words, keeping 56741 word types
2025-04-15 10:23:16,010 : INFO : PROGRESS: at sentence #140000, processed 3125257 words, kee

测试我们的模型

In [7]:
# 返回最不相识的
# 新版的gensim将词向量相关内容移动到.wv中
model.wv.doesnt_match("man woman child kitchen".split())

'kitchen'

In [8]:
# 我们的模型对细微的差异很敏感
# 如: 能够区分城市和国家
model.wv.doesnt_match("france england germany berlin".split())

'berlin'

In [9]:
model.wv.most_similar("awful")

[('terrible', 0.7680249214172363),
 ('atrocious', 0.7387546300888062),
 ('horrible', 0.728714108467102),
 ('abysmal', 0.7257044315338135),
 ('dreadful', 0.7020032405853271),
 ('horrendous', 0.6890418529510498),
 ('appalling', 0.6840807199478149),
 ('horrid', 0.6580556631088257),
 ('lousy', 0.6458923816680908),
 ('amateurish', 0.6203751564025879)]

### 第三部分

在这一部分, 我们尝试如何将训练好的词向量运用到监督学习中.

word2vec的词向量保存在wv.vectors中.其中行数为字典中的word number, 列数为训练模型时规定的num_feature

In [10]:
from gensim.models import Word2Vec
model = Word2Vec.load("300features_40minwords_10context")

print(type(model.wv.vectors))
print(model.wv.vectors.shape)

2025-04-15 10:24:15,220 : INFO : loading Word2Vec object from 300features_40minwords_10context
2025-04-15 10:24:15,261 : INFO : loading wv recursively from 300features_40minwords_10context.wv.* with mmap=None
2025-04-15 10:24:15,261 : INFO : setting ignored attribute cum_table to None
2025-04-15 10:24:15,435 : INFO : Word2Vec lifecycle event {'fname': '300features_40minwords_10context', 'datetime': '2025-04-15T10:24:15.435516', 'gensim': '4.2.0', 'python': '3.7.3 (default, Apr 24 2019, 15:29:51) [MSC v.1915 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'loaded'}


<class 'numpy.ndarray'>
(16731, 300)


可以查看特定单词的词向量

In [11]:
print(model.wv["flower"])
print(len(model.wv["flower"]))

[-4.28697392e-02  6.70673745e-03  3.93957943e-02  5.65489680e-02
 -2.30360273e-02 -1.49822399e-01  1.03891112e-01  1.18223391e-01
  9.99624059e-02 -1.14724427e-01  1.31973177e-01  4.82490994e-02
 -9.01374295e-02  1.98160633e-02  2.62181740e-02 -3.89472730e-02
  4.37283814e-02 -9.84854028e-02 -3.68942283e-02 -5.19021377e-02
 -2.66141426e-02  2.05458738e-02 -2.12340821e-02  7.90132117e-03
  1.15080968e-01 -4.47696261e-02  4.49908432e-03  4.93415445e-02
 -1.40867054e-01 -1.18495477e-02  1.07565798e-01 -9.33962092e-02
 -1.22588007e-02 -8.77175182e-02  2.36577261e-02  4.38936986e-02
  7.20364675e-02 -8.65962058e-02 -2.10110415e-02 -1.00770362e-01
 -5.97619787e-02  1.39087932e-02  7.94962198e-02  4.73498143e-02
 -4.44732569e-02 -2.96993665e-02  4.84491289e-02 -5.31749194e-03
 -5.31823514e-03  9.02162269e-02  1.09570071e-01  5.62805310e-03
 -6.96468204e-02 -2.58129574e-02 -9.10935253e-02  1.00488827e-01
 -1.96520630e-02  1.95702277e-02 -8.26002583e-02 -3.51381712e-02
 -4.74998541e-02  1.02703

### 从单词到段落: 尝试1,向量平均

我们可以将整个review的词向量简单的相加求平均, 用平均后向量来代表整个段落.

In [16]:
import numpy as np

# 将review转换为word vector
def makeFeatureVec(words, model, num_features):
    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed # index2word 
    index2word_set = set(model.wv.index_to_key)
    for word in words:
        if word in index2word_set:
            nwords += 1
            featureVec = np.add(featureVec, model.wv[word])
    
    featureVec = np.divide(featureVec, nwords)
    return featureVec

def getAvgFeatureVecs(reviews, model, num_features):
    reviewFeatureVecs = np.zeros((len(reviews),num_features), dtype="float32")
    for i, review in enumerate(reviews):
        if i % 1000 == 0:
            print(f"Review {i} of {len(reviews)}")
        reviewFeatureVecs[i] = makeFeatureVec(review, model, num_features)
    return reviewFeatureVecs

In [17]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

print("Creating average feature vecs for train reviews")
clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Creating average feature vecs for train reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 

In [18]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print("Fitting a random forest to labeled training data...")
forest = forest.fit( trainDataVecs, train["sentiment"] )

# Test & extract results 
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

d:\ANACONDA\lib\site-packages\sklearn\decomposition\online_lda.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps
d:\ANACONDA\lib\site-packages\sklearn\ensemble\gradient_boosting.py:32: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from ._gradient_boosting import predict_stages
d:\ANACONDA\lib\site-packages\sklea

Fitting a random forest to labeled training data...


d:\ANACONDA\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
d:\ANACONDA\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for addi

实验结果表明该算法比词袋模型的表现少几个百分点

### 从单词到段落: 尝试2,聚类

作者通过聚类算法(K-Means)将word2vec的词典聚类. 那么如何将review转换为vector呢? 作者模仿词袋模型, 将vector的纵坐标表示为"该review包含多少个第i个聚类中的单词".即: bag of words ----> bag of centrioids

In [23]:
from sklearn.cluster import KMeans
import time

start = time.time() # Start time

# 作者认为每个聚类中最多只有5个单词效果可能更好
word_vectors = model.wv.vectors
num_clusters = word_vectors.shape[0] // 5

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print("Time taken for K Means clustering: ", elapsed, "seconds.")

d:\ANACONDA\lib\site-packages\sklearn\cluster\k_means_.py:442: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  max_iter=max_iter, verbose=verbose)
d:\ANACONDA\lib\site-packages\sklearn\cluster\k_means_.py:442: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link fo

Time taken for K Means clustering:  1114.7100853919983 seconds.


In [24]:
# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.wv.index_to_key, idx ))
print(word_centroid_map)

{'the': 1988, 'and': 1707, 'a': 1847, 'of': 1648, 'to': 3159, 'is': 1079, 'it': 3071, 'in': 1878, 'i': 819, 'this': 3071, 'that': 215, 's': 1079, 'was': 440, 'as': 1773, 'with': 3126, 'for': 3195, 'movie': 3177, 'but': 913, 'film': 3177, 'you': 1004, 't': 435, 'on': 891, 'not': 1024, 'he': 2305, 'are': 1162, 'his': 1749, 'have': 1272, 'be': 207, 'one': 2612, 'all': 2571, 'at': 3165, 'they': 1004, 'by': 2086, 'who': 823, 'an': 1456, 'from': 3054, 'so': 1221, 'like': 2258, 'there': 2839, 'her': 783, 'or': 3044, 'just': 673, 'about': 1373, 'out': 2074, 'has': 3270, 'if': 1997, 'what': 1380, 'some': 3311, 'good': 1770, 'can': 633, 'more': 1611, 'when': 528, 'very': 2402, 'she': 2305, 'up': 1441, 'no': 2654, 'time': 3026, 'even': 1985, 'would': 911, 'my': 1132, 'which': 215, 'their': 821, 'story': 8, 'only': 3039, 'really': 673, 'see': 3175, 'had': 3270, 'were': 1162, 'well': 644, 'we': 1004, 'me': 819, 'than': 3202, 'much': 1016, 'bad': 1770, 'get': 767, 'been': 648, 'people': 2796, 'also'

处理review

In [25]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
     #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

In [26]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["review"].size, num_clusters), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for i, review in enumerate(clean_train_reviews):
    train_centroids[i] = create_bag_of_centroids( review, \
        word_centroid_map )

# Repeat for test reviews 
test_centroids = np.zeros(( test["review"].size, num_clusters), \
    dtype="float32" )

counter = 0
for i, review in enumerate(clean_test_reviews):
    test_centroids[i] = create_bag_of_centroids( review, \
        word_centroid_map )

训练模型

In [27]:
# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest may take a few minutes
print("Fitting a random forest to labeled training data...")
forest = forest.fit(train_centroids,train["sentiment"])
result = forest.predict(test_centroids)

# Write the test results 
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "BagOfCentroids.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


d:\ANACONDA\lib\site-packages\sklearn\ensemble\forest.py:489: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_store_unique_indices = np.zeros(y.shape, dtype=np.int)
d:\ANACONDA\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release